### Imports

In [1]:
import os
import torch
from pytorch3d.io import load_obj, save_obj
from pytorch3d.structures import Meshes, Pointclouds
from pytorch3d.utils import ico_sphere
from pytorch3d.ops import sample_points_from_meshes
import numpy as np
from tqdm.notebook import tqdm
%matplotlib notebook 
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['savefig.dpi'] = 80
mpl.rcParams['figure.dpi'] = 80

# Set the device
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

### Ground truth map mesh

In [2]:
verts, faces, aux = load_obj(os.path.join('../data/meshes/simple_cave_03.obj'))
# verts, faces, aux = load_obj(os.path.join('/home/ruslan/CTU/thirdparty/pytorch3d/docs/tutorials/dolphin.obj'))

# verts is a FloatTensor of shape (V, 3) where V is the number of vertices in the mesh
# faces is an object which contains the following LongTensors: verts_idx, normals_idx and textures_idx
# For this tutorial, normals and textures are ignored.
faces_idx = faces.verts_idx.to(device)
verts = verts.to(device)

# We construct a Meshes structure for the target mesh
mesh = Meshes(verts=[verts], faces=[faces_idx])

In [3]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

def compute_average_spacing(X, k=5):
    """
    Computes the average distance to nearest neighbours
    among a set of n points in the d-dimensional space.

    Arguments:
        X {numpy array} - the query points in an array of shape (n, d), 
                          where n is the number of points and d is the dimension.
    Returns:
        {float} - the average distance to nearest neighbours among the points
    """
    nbrs = NearestNeighbors(n_neighbors=k, algorithm='ball_tree').fit(X)
    distances, _ = nbrs.kneighbors(X)
    return np.mean(distances[:, 1:])  # distances to all k neighbours except itself

In [9]:
points = sample_points_from_meshes(mesh, 100000)

compute_average_spacing(points.cpu().numpy().squeeze(0), k=2)

0.4738367961178226

### Point cloud to compare

In [4]:
def plot_pointcloud(inputs, title=""):
    assert isinstance(inputs, Meshes) or isinstance(inputs, Pointclouds) or isinstance(inputs, torch.Tensor)
    if isinstance(inputs, Meshes):
        # Sample points uniformly from the surface of the mesh.
        points = sample_points_from_meshes(inputs, 5000)
    elif isinstance(inputs, Pointclouds):
        points = inputs.points_list()[0]
    elif isinstance(inputs, torch.Tensor):
        points = inputs
    x, y, z = points.clone().detach().cpu().squeeze().unbind(1)    
    fig = plt.figure(figsize=(5, 5))
    ax = Axes3D(fig)
    ax.scatter3D(x, z, -y)
    ax.set_xlabel('x')
    ax.set_ylabel('z')
    ax.set_zlabel('y')
    ax.set_title(title)
    ax.view_init(190, 30)
    plt.show()

In [20]:
torch.manual_seed(0)
points = sample_points_from_meshes(mesh, 5000)
# points = verts[None][:, torch.randint(verts.shape[0], (5000,)), :]
pcl = Pointclouds(points)

In [ ]:
plot_pointcloud(mesh)

In [ ]:
plot_pointcloud(pcl)

### Compare mesh to point cloud

In [ ]:
# Compare mesh and point cloud:
# https://pytorch3d.readthedocs.io/en/latest/modules/loss.html#pytorch3d.loss.point_mesh_edge_distance
from pytorch3d.loss import point_mesh_edge_distance, point_mesh_face_distance
from rpz_planning import point_face_distance_truncated
from rpz_planning import point_edge_distance_truncated
from rpz_planning import face_point_distance_truncated
from rpz_planning import edge_point_distance_truncated
from rpz_planning import chamfer_distance_truncated

In [ ]:
# point cloud is simply obtained from mesh vertices
with torch.no_grad():
    for s in range(5000, 50001, 5000):
        points = verts[None][:, torch.randint(verts.shape[0], (s,)), :]
        pcl = Pointclouds(points)
        # plot_pointcloud(pcl)
        print('Num sampled points:', s)
        print('EP:', edge_point_distance_truncated(meshes=mesh, pcls=pcl))
        print('PE:', point_edge_distance_truncated(meshes=mesh, pcls=pcl))
        print('EPPE:', point_mesh_edge_distance(meshes=mesh, pcls=pcl).sqrt())
        print('\n')

        print('FP:', face_point_distance_truncated(meshes=mesh, pcls=pcl))
        print('PF:', point_face_distance_truncated(meshes=mesh, pcls=pcl))
        print('FPPF:', point_mesh_face_distance(meshes=mesh, pcls=pcl).sqrt())
        print('-'*30)

In [ ]:
# point cloud is sampled from mesh surface
with torch.no_grad():
    for s in range(5000, 50001, 5000):
        points = sample_points_from_meshes(mesh, s)
        pcl = Pointclouds(points)
        # plot_pointcloud(pcl)
        print('Num sampled points:', s)
        print('EP:', edge_point_distance_truncated(meshes=mesh, pcls=pcl))
        print('PE:', point_edge_distance_truncated(meshes=mesh, pcls=pcl))
        print('EPPE:', point_mesh_edge_distance(meshes=mesh, pcls=pcl).sqrt())
        print('\n')

        print('FP:', face_point_distance_truncated(meshes=mesh, pcls=pcl))
        print('PF:', point_face_distance_truncated(meshes=mesh, pcls=pcl))
        print('FPPF:', point_mesh_face_distance(meshes=mesh, pcls=pcl).sqrt())
        print('-'*30)

### Using Trimesh library

In [ ]:
import trimesh

mesh = trimesh.load('../data/meshes/simple_cave_01.obj')

In [ ]:
from time import time

for s in range(1000, 10_001, 1000):
    t0 = time()
    # find the closest point on the mesh to each random point
    (closest_points,
    distances,
    triangle_id) = mesh.nearest.on_surface(mesh.vertices[np.random.choice(len(mesh.vertices), s)])
    print(f'For N vertices {s}, computation took {(time()-t0):.3f} [sec]')
    print(f'Mean distance: {distances.mean()}')

In [ ]:
pts = verts.detach().cpu()[torch.randint(verts.shape[0], (10000,)), :]
closest_points, distances, triangle_id = mesh.nearest.on_surface(pts)
print(distances.mean())